In [1]:
from sympy import *

In [2]:
sm, pr = symbols('sm, pr')
Dall = symbols('Dx, Dy, Dz')
Dx, Dy, Dz = Dall
nsrcx, nsrcy, nsrcz = symbols('nsrcx, nsrcy, nsrcz')
nobsAll = symbols('nobsx, nobsy, nobsz')
nobsx, nobsy, nobsz = nobsAll
CsRT0, CsRT1, CsRT2 = symbols('CsRT0, CsRT1, CsRT2')
srcb = [symbols('srcb' + str(i)) for i in range(3)]
bsrc_surf_curl = [[symbols('bsrc_surf_curl' + str(i) + str(j)) for j in range(3)] for i in range(3)]

In [3]:
r2 = Dx ** 2 + Dy ** 2 + Dz ** 2
invr = 1 / sqrt(r2)
invr3 = invr * invr * invr
Q1 = CsRT1 * invr
Q2 = CsRT0 * invr3
Q3 = CsRT2 * invr3
Q3nD = Q3 * (nsrcx * Dx + nsrcy * Dy + nsrcz * Dz)

In [4]:
e = [[[int((i - j) * (j - k) * (k - i) / 2) for k in range(3)]
    for j in range(3)] for i in range(3)]
kronecker = np.array([[1,0,0],[0,1,0],[0,0,1]])

In [15]:
Ks = [[[0 for i in range(3)] for j in range(3)] for k in range(3)]
for d_obs in range(3):
    for d_src in range(3):
        for b_src in range(3):
            Kval = 0
            for Ij in range(3):
                A = Q1 * e[Ij][d_obs][d_src]
                B = 0; 
                for It in range(3):
                    B += e[Ij][It][d_src] * Dall[It]
                Gimk = A + Q2 * Dall[d_obs] * B
                Kval += Gimk * bsrc_surf_curl[b_src][Ij]
            if d_obs == d_src:
                Kval += Q3nD * srcb[b_src]
            Ks[d_obs][b_src][d_src] = Kval

In [17]:
import pickle
pickle.dump(Ks, open('../tectosaur/data/sympyT.pkl', 'wb'))

In [23]:
Hs = [[[0 for i in range(3)] for j in range(3)] for k in range(3)]
for b_src in range(3):
    for d_src in range(3):
        disp_grad = [[diff(Ks[l][b_src][d_src], Dall[m]) for m in range(3)] for l in range(3)]
        strain = [[(disp_grad[l][m] + disp_grad[m][l]) / 2 for m in range(3)] for l in range(3)]
        strain_trace = sum([strain[d][d] for d in range(3)])
        lame_lambda = (2 * sm * pr) / (1 - 2 * pr)
        stress = [[lame_lambda * strain_trace * kronecker[l,m] + 2 * sm * strain[l][m] for m in range(3)] for l in range(3)]
        trac = [sum([stress[i][d] * nobsAll[d] for d in range(3)]) for i in range(3)]
        for d_obs in range(3):
            Hs[d_obs][b_src][d_src] = -trac[d_obs]

In [24]:
pickle.dump(Hs, open('../tectosaur/data/semiregularizedH.pkl', 'wb'))